# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step -1: Download dataset and unzip

In [ ]:
import os
import requests
import numpy as np
from tqdm import tqdm
from zipfile import ZipFile

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from datetime import datetime
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def download(url: str, fname: str):
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
            desc=fname,
            total=total,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)

In [ ]:
file_name = "traffic-signs-data.zip"
dest_name = "data_set"
url = "https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip"
if not os.path.exists(file_name):
    download(url, file_name)
with ZipFile(file=file_name) as zip_file:
    for file in tqdm(iterable=zip_file.namelist(), total=len(zip_file.namelist())):
        zip_file.extract(member=file, path=dest_name)

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file   = "data_set/train.p"
validation_file = "data_set/valid.p"
testing_file    = "data_set/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test,  y_test  = test['features'],  test['labels']

In [ ]:
indices = []
y_train_values = np.unique(y_train)

for index in y_train_values:
    indices.append(np.argwhere(y_train == index)[0][0])
    
fig1, axes = plt.subplots(ncols=len(y_train_values)//4, nrows=4, figsize=(15,5), constrained_layout=True)
axes = axes.flatten()

for image_index, axe in zip(indices, axes):
    axe.imshow(X_train[image_index+50])
    axe.set_title("class " + str(y_train[image_index]))
    axe.axis('off')

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

num_classes = 43
input_shape = (32, 32, 3)

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of validation examples
n_validation = X_valid.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = np.unique(y_train).shape[0]

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:
rows = 2
cols = 4
indices = np.random.randint(0, n_train, rows*cols)
fig1, f1_axes = plt.subplots(ncols=cols, nrows=rows, constrained_layout=True)
f1_axes = f1_axes.flatten()
for i, index in enumerate(indices):
    f1_axes[i].imshow(X_train[index])
    f1_axes[i].set_title(f'{y_train[index]}')
    f1_axes[i].axis('off')

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

I decided not to use augmentation it give better accuracy

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
datagen = ImageDataGenerator(
#    rotation_range=20,
#    horizontal_flip=True
)
### Feel free to use as many code cells as needed.

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)
y_test  = keras.utils.to_categorical(y_test,  num_classes)

### Model Architecture

### Callbacks
1. TensorBoard callback
2. Early stopping callbak

In [ ]:
# TensorBoard
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
# EarlyStopping
early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

### Model

| Layer | Type | Kernal | Features | Activation |
|:-:|:-:|:-:|:-:|:-:|
| Input | | 32x32 |
| C1 | Convolution | 5x5 | 6 | tanh |
| S2 | Sub-sampling | 2x2 | | |
| C3 | Convolution | 5x5 | 16 | tanh |
| S4 | Sub-sampling | 5x5 |
| C5 | Convolution | 5x5 | 120 | tanh |
| F6 | Fully Connected | | 84 | tanh |
| F7 | Fully Connected | | 10 | | Softmax |

In [ ]:
use_pretrained_model = False
model_path = 'models/lenet_no_agm_with_dropout75.h5'

#if(use_pretrained_model and os.path.exists(model_path)):
#    print("Pretrained model")
#    model = keras.models.load_model(model_path)
#else:
model = keras.Sequential(
        [
            layers.InputLayer(input_shape=input_shape),
            layers.Lambda(lambda x: tf.image.rgb_to_grayscale(x), name="RGB2Gray"),
            layers.Lambda(lambda x: (x / 255.0) + 0.5, name="Normalize"),
            layers.Conv2D(6, kernel_size=(5, 5), strides=1, name="C1", activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2), strides=2, name="S2"),
            layers.Conv2D(16, kernel_size=(5, 5), strides=1, name="C3", activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2), strides=2, name="S4"),
            layers.Conv2D(120, kernel_size=(5, 5), strides=1, name="C5", activation="relu"),
            layers.Flatten(), 
            layers.Dense(84, activation="relu", name="F6"),
            layers.Dropout(0.75),
            layers.Dense(num_classes, name="F7", activation="softmax"),
        ]
    )

model.summary()

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
batch_size = 128
epochs = 50

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])\

model.load_weights(model_path)

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
history = model.fit(x=datagen.flow(X_train, y_train, batch_size=batch_size),
          validation_data=datagen.flow(X_test,  y_test),
          steps_per_epoch=len(X_train) // batch_size,
          epochs=epochs,
          callbacks=[tensorboard_callback, early_stopping])

In [ ]:
def plot_history(history):
  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']
  loss     = history.history['loss']
  val_loss = history.history['val_loss']

  epoc_list = range(1, len(accuracy) + 1)

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
  ax1.plot(epoc_list, accuracy, 'b', label='Training acc')
  ax1.plot(epoc_list, val_accuracy, 'r', label='Validation acc')
  ax1.set_title('Training and validation accuracy')
  ax2.plot(epoc_list, loss, 'b', label='Training loss')
  ax2.plot(epoc_list, val_loss, 'r', label='Validation loss')
  ax2.set_title('Training and validation loss')
  fig.legend()

In [ ]:
plot_history(history)

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
X_valid.shape, model.layers[0].input_shape

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
score = model.evaluate(X_valid, y_valid, verbose=0)
print("Valid loss:", score[0])
print("Valid accuracy:", score[1])

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
test_images  = ["TGS_01.jpg", "TGS_02.jpg", "TGS_03.jpg",
                "TGS_04.jpg", "TGS_05.jpg", "TGS_07.jpg",
                "TGS_08.jpg"]
test_labels = np.array([18,  5, 35, 38, 17, 37, 13], dtype=np.float32)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

In [ ]:
from matplotlib.pyplot import imread

fig1, axes = plt.subplots(ncols=len(test_images), nrows=1, figsize=(15,5), constrained_layout=True)
axes = axes.flatten()

real_time_images = np.zeros((len(test_images), 32, 32, 3))
for index, (image, axe) in enumerate(zip(test_images, axes)):
    image_ = imread("data_set/testing_images/" + image)
    real_time_images[index] = image_
    axe.imshow(image_)
    axe.axis('off')

In [ ]:
real_time_images.shape, test_labels.shape

In [ ]:
((34799, 32, 32, 3), (34799, 43))

In [ ]:
score = model.evaluate(real_time_images, test_labels, verbose=0)
print("Valid loss:", score[0])
print("Valid accuracy:", score[1])

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
score = model.predict(real_time_images)
sorted_indices = np.argsort(score)

In [ ]:
pos = np.arange(0, 5)

for image, score_value, so in zip(real_time_images, score, sorted_indices):
    fig, (ax0, ax1) = plt.subplots(ncols=2, nrows=1, figsize=(10,3), constrained_layout=True)
    ax0.imshow(image.astype(np.uint8))
    ax0.axis('off')
    test_names = so[-5:]
    ax1.barh(pos, score_value[so][-5:], align='center', height=0.5, tick_label=test_names)

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
test_image = X_valid[150]

layer_outputs    = [layer.output for layer in model.layers[:6]]
activation_model = keras.Model(inputs=model.input, outputs=layer_outputs)

activations = activation_model.predict(np.array([test_image]))

In [ ]:
# predicting images
x = np.expand_dims(test_image, axis=0)
images = np.vstack([x])
classes = model.predict_classes(images, batch_size=10)

In [ ]:
layer_activation = activations[2]
print(layer_activation.shape)

In [ ]:
plt.imshow(test_image, cmap="gray")
plt.axis("off")

In [ ]:
import cv2

count = layer_activation.shape[-1]
fig1, f1_axes = plt.subplots(ncols=count//2, nrows=2, figsize=(5, 3), constrained_layout=True)
f1_axes = f1_axes.flatten()
for index in range(count):
  # i = np.random.randint(0, 6)
  img = layer_activation[0,:,:,index]
  out = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  f1_axes[index].axis('off')
  f1_axes[index].imshow(out, cmap="gray")

In [ ]:
layer_activation = activations[4]
print(layer_activation.shape)

count = layer_activation.shape[-1]
fig1, f1_axes = plt.subplots(ncols=count//4, nrows=4, figsize=(5,4), constrained_layout=True)
f1_axes = f1_axes.flatten()
for index in range(count):
  # i = np.random.randint(0, 6)
  img = layer_activation[0,:,:,index]
  out = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  f1_axes[index].axis('off')
  f1_axes[index].imshow(out, cmap="gray")